### 1. Comma.ai model

#### Model

In [ ]:
def get_model(time_len=1):
    ch, row, col = 3, 160, 320  # camera format
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,input_shape=(row, col,ch),output_shape=(row, col,ch)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse")

    return model

#### Sanity checks

In [ ]:
sanity_batch_data = np.zeros((3,160,320,3))
sanity_batch_steering = np.zeros(3)

sanity_batch_data[0] = cv2.cvtColor(cv2.imread(data_path+data_log.iloc[54]['center'].strip()),cv2.COLOR_BGR2RGB)
sanity_batch_data[1] = cv2.cvtColor(cv2.imread(data_path+data_log.iloc[0]['center'].strip()),cv2.COLOR_BGR2RGB)
sanity_batch_data[2] = cv2.cvtColor(cv2.imread(data_path+data_log.iloc[154]['center'].strip()),cv2.COLOR_BGR2RGB)

sanity_batch_steering[0] = data_log.iloc[54]['steering']
sanity_batch_steering[1] = data_log.iloc[0]['steering']
sanity_batch_steering[2] = data_log.iloc[154]['steering']

print(sanity_batch_steering)

gs1 = gridspec.GridSpec(1,3)
gs1.update(wspace=0.01, hspace=0.02) # set the spacing between axes. 
plt.figure(figsize=(12,12))
for i in range(3):
    ax1 = plt.subplot(gs1[i])
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])
    ax1.set_aspect('equal')
    img = sanity_batch_data[i]
    plt.subplot(1,3,i+1)
    plt.imshow(img)
    plt.axis('off')

plt.show()

#### Model Summary

In [ ]:
model = get_model()
print(model.summary())

#### Train the model

In [ ]:
model.fit(sanity_batch_data,sanity_batch_steering,nb_epoch=20)

#### Test the model

In [ ]:
model.evaluate(sanity_batch_data,sanity_batch_steering)
model.predict(sanity_batch_data)

#### Reason for discarding

The model's loss function is not going to zero i.e. not converging to zero and also the loss is oscillating between high and low loss even after 50 epochs